# Sample Implementation

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
# First, import TF and get its version.
import tensorflow as tf
tf_version = tf.__version__

# Check if version >=2.0.0 is used
if not tf_version.startswith('2.'):
    print('WARNING: TensorFlow >= 2.0.0 will be used in this course.\nYour version is {}'.format(tf_version) + '.\033[0m')
else:
    print('OK: TensorFlow >= 2.0.0' + '.\033[0m')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras import utils

import wandb
from wandb.keras import WandbCallback

wandb.login()
project_id = 'wandb-callbacks-demo'

%matplotlib inline

# fix random seed for reproducibility
np.random.seed(42)

# GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Define the dataset

In [ ]:
def show_imgs(X):
    plt.figure(1)
    k = 0
    for i in range(0,5):
        for j in range(0,5):
            plt.subplot2grid((5,5),(i,j))
            plt.imshow(X[k], cmap='gray')
            k = k+1
            plt.axis('off')
    # show the plot
    plt.show()
    
# Load data & split data between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
CLASS_NAMES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

show_imgs(X_train)

In [ ]:
# Normalize input
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

n_classes = 10
# Conversion to class vectors
Y_train = utils.to_categorical(y_train, n_classes)
Y_test = utils.to_categorical(y_test, n_classes)

## Sample implementation

In [ ]:
import sys
sys.path.append("./..")

from wandb_callbacks.tensorflow import ActivationCallback, DeadReluCallback, GRADCamCallback
from wandb_callbacks.utils import get_samples_for_activation

In [ ]:
D = X_train.shape[1:]

# initialize wandb with your project name and optionally with configutations.
run = wandb.init(project=project_id,
                 config={
                     "learning_rate": 0.005,
                     "epochs": 30,
                     "batch_size": 64,
                     "activation": 'relu',
                     "loss_function": "categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "CIFAR-10"})
config = wandb.config

def create_model(config):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), input_shape=D, activation='relu', kernel_initializer='he_normal'))
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(256, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(n_classes, activation='softmax'))
    
    model.compile(loss=config.loss_function, 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), 
                  metrics=['accuracy'])
    
    return model

tf.keras.backend.clear_session()
model = create_model(config)
model.summary()

In [ ]:
# samples to predict and use for visualisation
sample_images, sample_labels, sample_labels_enc = get_samples_for_activation(CLASS_NAMES, X_test, Y_test)

# default wandb callback to log training process and sample images
wandb_callback = WandbCallback(data_type='image', 
                               training_data=(sample_images, sample_labels_enc), 
                               labels=CLASS_NAMES, 
                               log_weights=True,
                               log_gradients=True)

# activation callback to visualize the activtion of one layer
act_callback = ActivationCallback(validation_data=(sample_images, sample_labels),
                                  layer_name='conv2d', 
                                  log_frequency=10)  # log only every 10 epochs

# keeps track of the number of dead relus within the network
relu_callback = DeadReluCallback(sample_images, 
                                 log_frequency=1, 
                                 verbose=False)

# creates a GRADCam for the given sample images and the given layer
cam_callback = GRADCamCallback(sample_images, 
                               layer_name='conv2d_3', 
                               log_frequency=1)

# train the model with the callbacks
log = model.fit(X_train, Y_train, 
                epochs=config.epochs, 
                batch_size=config.batch_size,
                validation_data=(X_test, Y_test),
                callbacks=[wandb_callback, act_callback, relu_callback, cam_callback])